In [ ]:
%pip install yfinance
%pip install matplotlib
%pip install pandas numpy
%pip install --upgrade sympy torch
%pip install torch torchvision torchaudio

# 3. Restart runtime again

# 4. Now import and run your code
import torch
print(torch.__version__)

# BASELINE NBEATS MODEL

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# ==================== IMPROVED NBEATS COMPONENTS ====================

class NBeatsBlock(nn.Module):
    """Single NBEATS block with proper basis functions"""
    def __init__(self, input_size, theta_size, horizon, n_layers=4, layer_size=256,
                 basis_function=None, share_weights=False):
        super().__init__()
        self.input_size = input_size
        self.theta_size = theta_size
        self.horizon = horizon
        self.basis_function = basis_function

        # Fully connected stack with layer normalization
        layers = []
        layers.append(nn.Linear(input_size, layer_size))
        layers.append(nn.ReLU())

        for _ in range(n_layers - 1):
            layers.append(nn.Linear(layer_size, layer_size))
            layers.append(nn.ReLU())

        self.fc_stack = nn.Sequential(*layers)

        # Theta layers for backcast and forecast
        self.theta_b = nn.Linear(layer_size, theta_size, bias=False)
        self.theta_f = nn.Linear(layer_size, theta_size, bias=False)

        # If no basis function provided, use generic linear basis
        if basis_function is None:
            self.backcast_basis = nn.Linear(theta_size, input_size)
            self.forecast_basis = nn.Linear(theta_size, horizon)

    def forward(self, x):
        # x: [batch_size, input_size]
        h = self.fc_stack(x)

        theta_b = self.theta_b(h)  # [batch_size, theta_size]
        theta_f = self.theta_f(h)  # [batch_size, theta_size]

        if self.basis_function is not None:
            # Interpretable mode (trend/seasonality)
            backcast = self.basis_function(theta_b, self.input_size)
            forecast = self.basis_function(theta_f, self.horizon)
        else:
            # Generic mode with proper linear projection
            backcast = self.backcast_basis(theta_b)
            forecast = self.forecast_basis(theta_f)

        return backcast, forecast


class TrendBasis(nn.Module):
    """Polynomial trend basis function"""
    def __init__(self, degree=3):
        super().__init__()
        self.degree = degree

    def forward(self, theta, length):
        # theta: [batch_size, degree+1]
        batch_size = theta.size(0)
        T = torch.arange(length, dtype=torch.float32, device=theta.device) / length
        T = T.unsqueeze(0).repeat(batch_size, 1)  # [batch_size, length]

        # Create polynomial basis [1, t, t^2, t^3, ...]
        basis = torch.stack([T**i for i in range(self.degree + 1)], dim=2)  # [batch_size, length, degree+1]
        output = torch.einsum('btd,bd->bt', basis, theta)  # [batch_size, length]

        return output


class SeasonalityBasis(nn.Module):
    """Fourier series basis function for seasonality"""
    def __init__(self, harmonics=10):
        super().__init__()
        self.harmonics = harmonics

    def forward(self, theta, length):
        # theta: [batch_size, 2*harmonics]
        batch_size = theta.size(0)
        T = torch.arange(length, dtype=torch.float32, device=theta.device) / length
        T = T.unsqueeze(0).repeat(batch_size, 1)  # [batch_size, length]

        # Create Fourier basis [cos(2πt), sin(2πt), cos(4πt), sin(4πt), ...]
        basis = []
        for i in range(1, self.harmonics + 1):
            basis.append(torch.cos(2 * np.pi * i * T))
            basis.append(torch.sin(2 * np.pi * i * T))

        basis = torch.stack(basis, dim=2)  # [batch_size, length, 2*harmonics]
        output = torch.einsum('btd,bd->bt', basis, theta)  # [batch_size, length]

        return output


class NBeatsStack(nn.Module):
    """Stack of NBEATS blocks with proper residual connections"""
    def __init__(self, input_size, horizon, n_blocks=3, n_layers=4, layer_size=256,
                 stack_type='generic', share_weights=False):
        super().__init__()
        self.input_size = input_size
        self.horizon = horizon
        self.stack_type = stack_type

        # Define basis function and theta size based on stack type
        if stack_type == 'trend':
            degree = 3
            basis_function = TrendBasis(degree=degree)
            theta_size = degree + 1
        elif stack_type == 'seasonality':
            harmonics = int(input_size / 2)  # Adaptive harmonics
            basis_function = SeasonalityBasis(harmonics=harmonics)
            theta_size = 2 * harmonics
        else:  # generic
            basis_function = None
            theta_size = max(input_size // 4, horizon // 4)  # Smaller theta size

        # Create blocks
        self.blocks = nn.ModuleList([
            NBeatsBlock(input_size, theta_size, horizon, n_layers, layer_size,
                       basis_function, share_weights)
            for _ in range(n_blocks)
        ])

    def forward(self, x):
        # x: [batch_size, input_size]
        residual = x
        forecast = torch.zeros(x.size(0), self.horizon, device=x.device)

        for block in self.blocks:
            backcast, block_forecast = block(residual)
            residual = residual - backcast  # Remove explained signal
            forecast = forecast + block_forecast  # Accumulate forecast

        return forecast, residual  # Return both forecast and final residual


class NBEATS(nn.Module):
    """
    Full NBEATS model with multiple stacks and doubly residual stacking
    """
    def __init__(self, input_size, horizon, stack_types=['trend', 'seasonality'],
                 n_blocks=[3, 3], n_layers=4, layer_size=256, share_weights=False):
        super().__init__()
        self.input_size = input_size
        self.horizon = horizon

        assert len(stack_types) == len(n_blocks), "stack_types and n_blocks must have same length"

        self.stacks = nn.ModuleList([
            NBeatsStack(input_size, horizon, n_blocks[i], n_layers, layer_size,
                       stack_types[i], share_weights)
            for i in range(len(stack_types))
        ])

    def forward(self, x):
        # x: [batch_size, input_size]
        residual = x
        forecast = torch.zeros(x.size(0), self.horizon, device=x.device)

        # Doubly residual stacking
        for stack in self.stacks:
            stack_forecast, residual = stack(residual)
            forecast = forecast + stack_forecast

        return forecast


# ==================== IMPROVED DATASET ====================

class TimeSeriesDataset(Dataset):
    """Custom dataset for time series forecasting with proper indexing"""
    def __init__(self, data, input_size, horizon):
        self.data = torch.FloatTensor(data)
        self.input_size = input_size
        self.horizon = horizon

    def __len__(self):
        return len(self.data) - self.input_size - self.horizon + 1

    def __getitem__(self, idx):
        x = self.data[idx:idx + self.input_size]
        y = self.data[idx + self.input_size:idx + self.input_size + self.horizon]
        return x, y


# ==================== TRAINING WITH EARLY STOPPING ====================

class EarlyStopping:
    """Early stopping to prevent overfitting"""
    def __init__(self, patience=10, min_delta=1e-4):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss > self.best_loss - self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = val_loss
            self.counter = 0


def train_epoch(model, dataloader, optimizer, criterion, device):
    model.train()
    total_loss = 0

    for x, y in dataloader:
        x, y = x.to(device), y.to(device)

        optimizer.zero_grad()
        output = model(x)
        loss = criterion(output, y)
        loss.backward()

        # Gradient clipping to prevent exploding gradients
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        optimizer.step()
        total_loss += loss.item()

    return total_loss / len(dataloader)


def validate(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0

    with torch.no_grad():
        for x, y in dataloader:
            x, y = x.to(device), y.to(device)
            output = model(x)
            loss = criterion(output, y)
            total_loss += loss.item()

    return total_loss / len(dataloader)


# ==================== MAIN EXECUTION ====================

def main():
    # Device configuration
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")

    # Download and prepare data

    ticker = 'GOOG'
    print(f"\nDownloading {ticker} stock data...")
    data = yf.download(ticker, start='2015-01-01', end='2023-12-31', auto_adjust=True)

    # Flatten MultiIndex if present
    if isinstance(data.columns, pd.MultiIndex):
        data.columns = data.columns.get_level_values(0)

    # Use Close price
    prices = data['Close'].values

    # Normalize using z-score normalization
    mean, std = prices.mean(), prices.std()
    prices_normalized = (prices - mean) / std

    print(f"Data shape: {prices.shape}")
    print(f"Price range: ${prices.min():.2f} - ${prices.max():.2f}")
    print(f"Mean: ${mean:.2f}, Std: ${std:.2f}")

    # Split data: 70% train, 15% validation, 15% test
    train_size = int(0.7 * len(prices_normalized))
    val_size = int(0.15 * len(prices_normalized))

    train_data = prices_normalized[:train_size]
    val_data = prices_normalized[train_size:train_size + val_size]
    test_data = prices_normalized[train_size + val_size:]

    print(f"Train: {len(train_data)}, Val: {len(val_data)}, Test: {len(test_data)}")

    # Hyperparameters - ADJUSTED TO PREVENT OVERFITTING
    INPUT_SIZE = 60      # lookback window
    HORIZON = 20         # forecast horizon
    BATCH_SIZE = 32      # Smaller batch size
    EPOCHS = 100         # More epochs with early stopping
    LEARNING_RATE = 1e-3

    # Create datasets
    train_dataset = TimeSeriesDataset(train_data, INPUT_SIZE, HORIZON)
    val_dataset = TimeSeriesDataset(val_data, INPUT_SIZE, HORIZON)
    test_dataset = TimeSeriesDataset(test_data, INPUT_SIZE, HORIZON)

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

    print(f"\nTraining samples: {len(train_dataset)}")
    print(f"Validation samples: {len(val_dataset)}")
    print(f"Test samples: {len(test_dataset)}")

    # Create improved model with better architecture
    model = NBEATS(
        input_size=INPUT_SIZE,
        horizon=HORIZON,
        stack_types=['trend', 'seasonality'],
        n_blocks=[2, 2],      # Reduced blocks
        n_layers=4,
        layer_size=128        # Reduced layer size
    ).to(device)

    total_params = sum(p.numel() for p in model.parameters())
    print(f"\n🤖 Model created with {total_params:,} parameters")

    # Loss and optimizer with weight decay (L2 regularization)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-5)

    # Learning rate scheduler
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min',
                                                      factor=0.5, patience=5)

    # Early stopping
    early_stopping = EarlyStopping(patience=15, min_delta=1e-5)

    # Training loop
    print("\nStarting training...")
    train_losses = []
    val_losses = []
    best_val_loss = float('inf')

    for epoch in range(EPOCHS):
        train_loss = train_epoch(model, train_loader, optimizer, criterion, device)
        val_loss = validate(model, val_loader, criterion, device)

        train_losses.append(train_loss)
        val_losses.append(val_loss)

        # Learning rate scheduling
        scheduler.step(val_loss)

        # Save best model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), '/tmp/best_nbeats_model.pt')

        if (epoch + 1) % 10 == 0:
            print(f"Epoch [{epoch+1}/{EPOCHS}] - Train: {train_loss:.6f}, Val: {val_loss:.6f}")

        # Early stopping
        early_stopping(val_loss)
        if early_stopping.early_stop:
            print(f"\n⚠️  Early stopping triggered at epoch {epoch+1}")
            break

    # Load best model
    model.load_state_dict(torch.load('/tmp/best_nbeats_model.pt'))
    print("\nTraining complete! Loaded best model.")

    # Evaluate on test set
    test_loss = validate(model, test_loader, criterion, device)
    print(f"Test Loss: {test_loss:.6f}")

    # Plot training curves
    plt.figure(figsize=(10, 5))
    plt.plot(train_losses, label='Train Loss', alpha=0.8)
    plt.plot(val_losses, label='Validation Loss', alpha=0.8)
    plt.xlabel('Epoch')
    plt.ylabel('MSE Loss')
    plt.title('NBEATS Training Progress (Improved)')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.yscale('log')  # Log scale to see details
    plt.show()

    # Make predictions on test set
    model.eval()
    all_predictions = []
    all_targets = []

    with torch.no_grad():
        for x, y in test_loader:
            x = x.to(device)
            pred = model(x).cpu().numpy()
            all_predictions.append(pred)
            all_targets.append(y.numpy())

    predictions = np.concatenate(all_predictions, axis=0)
    targets = np.concatenate(all_targets, axis=0)

    # Denormalize predictions
    predictions_denorm = predictions * std + mean
    targets_denorm = targets * std + mean

    # Compute metrics
    mae = np.mean(np.abs(targets_denorm - predictions_denorm))
    rmse = np.sqrt(np.mean((targets_denorm - predictions_denorm) ** 2))
    mape = np.mean(np.abs((targets_denorm - predictions_denorm) / targets_denorm)) * 100

    print(f"\nTest Set Performance:")
    print(f"  MAE:  ${mae:.2f}")
    print(f"  RMSE: ${rmse:.2f}")
    print(f"  MAPE: {mape:.2f}%")

    # Plot sample predictions
    plt.figure(figsize=(15, 6))

    # Show first 5 predictions
    for i in range(min(5, len(predictions_denorm))):
        plt.subplot(1, 5, i+1)
        plt.plot(targets_denorm[i], 'b-', label='Actual', linewidth=2)
        plt.plot(predictions_denorm[i], 'r--', label='Predicted', linewidth=2)
        plt.title(f'Sample {i+1}')
        plt.xlabel('Time Step')
        plt.ylabel('Price ($)')
        if i == 0:
            plt.legend()
        plt.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

    # Make future forecast
    model.eval()
    with torch.no_grad():
        last_window = torch.FloatTensor(test_data[-INPUT_SIZE:]).unsqueeze(0).to(device)
        future_prediction = model(last_window).cpu().numpy()[0]
    future_prediction_denorm = future_prediction * std + mean

    print(f"\nFuture forecast for next {HORIZON} days:")
    print(future_prediction_denorm)

    # Plot future forecast
    plt.figure(figsize=(12, 6))
    historical = test_data[-100:] * std + mean
    plt.plot(range(len(historical)), historical, 'b-', label='Historical', linewidth=2)
    plt.plot(range(len(historical), len(historical) + HORIZON),
             future_prediction_denorm, 'r--', label='Forecast', linewidth=2, marker='o')
    plt.xlabel('Time')
    plt.ylabel('Price ($)')
    plt.title(f'{ticker} Stock Price Forecast (Improved Model)')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()

    return model, train_losses, val_losses

# Run the main function
if __name__ == "__main__":
    model, train_losses, test_losses = main()